In [1]:
println(s"Current spark version is ${spark.version}")

Intitializing Scala interpreter ...

Spark Web UI available at http://36de47ca6b01:4040
SparkContext available as 'sc' (version = 3.0.0, master = local[*], app id = local-1595854352173)
SparkSession available as 'spark'


Current spark version is 3.0.0


In [2]:
import org.apache.spark.sql.types.{StructType, StructField, IntegerType, LongType, StringType}

val dataSchema = new StructType()
    .add("target", IntegerType)
    .add("id", LongType)
    .add("raw_timestamp", StringType)
    .add("query_status", StringType)
    .add("author", StringType)
    .add("tweet", StringType)

val dataPath= "/home/jovyan/data/training.1600000.processed.noemoticon.csv"

val raw_sentiment = spark.read
    .format("csv")
    .option("header",false)
    .schema(dataSchema)
    .load(dataPath)
    .selectExpr("(case when target=4 then 1 else 0 end) as label","tweet")

raw_sentiment.groupBy($"label").count.show

+-----+------+
|label| count|
+-----+------+
|    1|800000|
|    0|800000|
+-----+------+



import org.apache.spark.sql.types.{StructType, StructField, IntegerType, LongType, StringType}
dataSchema: org.apache.spark.sql.types.StructType = StructType(StructField(target,IntegerType,true), StructField(id,LongType,true), StructField(raw_timestamp,StringType,true), StructField(query_status,StringType,true), StructField(author,StringType,true), StructField(tweet,StringType,true))
dataPath: String = /home/jovyan/data/training.1600000.processed.noemoticon.csv
raw_sentiment: org.apache.spark.sql.DataFrame = [label: int, tweet: string]


In [7]:
import org.apache.spark.ml.{Pipeline, PipelineModel}
// import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row

val tokenizer = new Tokenizer()
    .setInputCol("tweet")
    .setOutputCol("words")

val hashingTF = new HashingTF()
    .setNumFeatures(1000)
    .setInputCol(tokenizer.getOutputCol)
    .setOutputCol("features")

// RandomForest
val rf = new RandomForestClassifier()
  .setLabelCol("label")
  .setFeaturesCol("features")
  .setNumTrees(10)
// val lr = new LogisticRegression()
//     .setMaxIter(10)
//     .setRegParam(0.001)

val pipeline = new Pipeline()
  .setStages(Array(tokenizer, hashingTF, rf))


import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row
tokenizer: org.apache.spark.ml.feature.Tokenizer = tok_df45b2192b6b
hashingTF: org.apache.spark.ml.feature.HashingTF = HashingTF: uid=hashingTF_f553313f3147, binary=false, numFeatures=1000
rf: org.apache.spark.ml.classification.RandomForestClassifier = rfc_f6a776d80ec9
pipeline: org.apache.spark.ml.Pipeline = pipeline_fa230c4baa13


In [8]:
val model = pipeline.fit(raw_sentiment)

model: org.apache.spark.ml.PipelineModel = pipeline_fa230c4baa13


In [9]:
model.write.overwrite().save("/home/jovyan/models/spark-ml-model")

In [10]:
val sameModel = PipelineModel.load("/home/jovyan/models/spark-ml-model")

sameModel: org.apache.spark.ml.PipelineModel = pipeline_fa230c4baa13


In [11]:
val predictionsDF = sameModel.transform(raw_sentiment)

predictionsDF.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|               tweet|               words|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    0|@switchfoot http:...|[@switchfoot, htt...|(1000,[10,21,81,1...|[4.69923926724613...|[0.46992392672461...|       1.0|
|    0|is upset that he ...|[is, upset, that,...|(1000,[121,193,20...|[4.82054962490642...|[0.48205496249064...|       1.0|
|    0|@Kenichan I dived...|[@kenichan, i, di...|(1000,[17,185,188...|[5.25858516766473...|[0.52585851676647...|       0.0|
|    0|my whole body fee...|[my, whole, body,...|(1000,[191,330,44...|[5.19676997473920...|[0.51967699747392...|       0.0|
|    0|@nationwideclass ...|[@nationwideclass...|(1000,[32,162,166...|[4.96198213089592...|[0.49619821308959...|       1.0|
|    0|@

predictionsDF: org.apache.spark.sql.DataFrame = [label: int, tweet: string ... 5 more fields]


In [12]:
// predictionsDF.schema
// predictionsDF.describe()
// predictionsDF.stat
predictionsDF.printSchema()

root
 |-- label: integer (nullable = false)
 |-- tweet: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [13]:
import org.apache.spark.sql.functions._

val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))

import org.apache.spark.sql.functions._
getProbability: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$5087/0x00000008419df840@7f9ad6cb,DoubleType,List(Some(class[value[0]: vector])),None,false,true)


In [14]:
predictionsDF.select(getProbability($"probability").alias("clean_probability")).show

+-------------------+
|  clean_probability|
+-------------------+
| 0.5300760732753866|
| 0.5179450375093577|
|0.47414148323352634|
|0.48032300252607935|
| 0.5038017869104074|
|  0.493523549682455|
|0.49332751297247723|
| 0.5923063170518399|
|  0.505078305345407|
| 0.4909974038751022|
| 0.5139034718036193|
|0.47529570246457664|
|0.49704755116204347|
| 0.5132201526338107|
|0.48574248379315543|
| 0.4285555594179017|
|  0.493523549682455|
| 0.5139034718036193|
| 0.5487794397402259|
| 0.5300760732753866|
+-------------------+
only showing top 20 rows

